In [ ]:
!pip install tf-nightly-gpu-2.0-preview==2.0.0.dev20190218

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install -q tensorflow-gpu==2.0.0-beta1
!pip install tensorflow_datasets
import tensorflow_datasets as tfds
import tensorflow as tf


In [ ]:
!pip install -q tensorflow-gpu==2.0.0-nightly-gpu-dev040404

In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()


In [ ]:
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True,
                          as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']


In [ ]:
tokenizer = info.features['text'].encoder

print ('Vocabulary size: {}'.format(tokenizer.vocab_size))


In [ ]:
sample_string = 'TensorFlow is cool.'

tokenized_string = tokenizer.encode(sample_string)
print ('Tokenized string is {}'.format(tokenized_string))

original_string = tokenizer.decode(tokenized_string)
print ('The original string: {}'.format(original_string))

assert original_string == sample_string


In [ ]:
for ts in tokenized_string:
    print ('{} ----> {}'.format(ts, tokenizer.decode([ts])))

    

In [ ]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64


In [ ]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.padded_batch(BATCH_SIZE, train_dataset.output_shapes)

test_dataset = test_dataset.padded_batch(BATCH_SIZE, test_dataset.output_shapes)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout,Dense,LSTM

In [ ]:
# from keras.models import Sequential
# from keras.layers import Dropout,Dense,LSTM,CuDNNLSTM,Embedding,Bidirectional

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(tokenizer.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [ ]:
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset)


In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))


In [ ]:
def pad_to_size(vec, size):
    zeros = [0] * (size - len(vec))
    vec.extend(zeros)
    return vec

def sample_predict(sentence, pad):
    tokenized_sample_pred_text = tokenizer.encode(sample_pred_text)

    if pad:
        tokenized_sample_pred_text = pad_to_size(tokenized_sample_pred_text, 64)

    predictions = model.predict(tf.expand_dims(tokenized_sample_pred_text, 0))

    return (predictions)


In [ ]:

# predict on a sample text without padding.

sample_pred_text = ('The movie was ok. The animation and the graphics '
                    'were not good. I would recommend this movie.')
predictions = sample_predict(sample_pred_text, pad=False)
print (predictions)


In [ ]:
# predict on a sample text with padding

sample_pred_text = ('The movie not good. The animation and the graphics '
                    'were out of this world. I would recommend this movie.')
predictions = sample_predict(sample_pred_text, pad=True)
print (predictions)


In [ ]:
plot_graphs(history, 'accuracy')


In [ ]:
plot_graphs(history, 'loss')
